In [4]:
#STEPS TO DO:
#
#Step 1 : Open the attachment
#         Load today's file
#Step 2 : Auto filter & format handling (done in Excel itself)
#Step 3 : Copy all the order id's in one more column              xxx
#Step 4 : Convert ITEM_ID to fraction (as string)
#Step 5 : Remove rows with "duplicate" in ORDER NUMBER and ITEM_ID
#Step 6 : Remove the Order Numbers matched with Order No in Need to Remove - Order.. file
#Step 7 : Do a concatenate with ORDER NO and VARIANT
#         Create OV column
#Step 8 : Remove duplicates from oder id's and OV columns
#Step 9 : Load previous report and repeat the steps as done in today's file if not already performed
#Step 10: VLOOKUP-style comparison
#         Remove all the order id's by doing vlookup with the sheet shared by TOPS
#         Do a VLOOKUP with Order variant combo with last shared file
#Step 11: Mark all the NA's as NO and all the orders as YES
#Step 12: Creating Month and Day-Month Columns and sorting them
#Step 13: Create pivot table
#Step 14: Save results to new Excel file

In [5]:
#NOTE - 1. Run all cells at once and then, in pop-up window, browse and select first the current day's report file then the previous report file.
#TIP - (Rows x Columns) are shown below each table to see the changes made

In [6]:
#Importing Necessary libraries
import pandas as pd
import numpy as np
from fractions import Fraction
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
import tkinter as tk
from tkinter import filedialog, Tk
from tkinter.filedialog import askopenfilename
import warnings
warnings.filterwarnings("ignore")

In [7]:
#Shows the place where the code is stored
import os
print(os.getcwd())

C:\Users\aryan\Desktop\Aryan\BITS Pilani\RIL SIP\06-16-2025


## Importing the File to Work on

In [9]:
#Browse the file from system and read it
file_path = askopenfilename(
    title="Select a data file",
    filetypes=[("Data files", "*.xlsx *.xls *.csv"), ("All files", "*.*")])

# Detect file extension and load accordingly
if file_path.endswith('.csv'):
    df = pd.read_csv(file_path)
elif file_path.endswith(('.xls', '.xlsx')):
    df = pd.read_excel(file_path)
else:
    raise ValueError("Unsupported file format")

print("File loaded successfully:")
df.head()

File loaded successfully:


,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,Prepaid,NaN
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,Prepaid,pending
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,Prepaid,success
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,Prepaid,success
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,Prepaid,success


In [10]:
#Converting ITEM_ID into fraction
df["ITEM_ID"] = df["ITEM_ID"].apply(lambda x: str(Fraction(x).limit_denominator()) if pd.notna(x) else x)
df

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,Prepaid,NaN
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,Prepaid,pending
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,Prepaid,success
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,Prepaid,success
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,Prepaid,success
...,...,...,...,...,...,...,...
7019,Return Exceptions,IN_PROGRESS,2025-06-12 12:18:19,RT183950955,700877196005,COD,NaN
7020,Return Exceptions,IN_PROGRESS,2025-06-12 16:30:09,RT183980671,701449142003,Prepaid,NaN
7021,Return Exceptions,IN_PROGRESS,2025-06-12 22:23:50,RT184041402,701558260004,Prepaid,NaN
7022,Return Exceptions,IN_PROGRESS,2025-06-12 14:40:38,RT183978459,700965211001,Prepaid,NaN


In [11]:
#Removing the rows where ORDER NUMBER contains the word "duplicate"
#TIP - What the code does is keep the rows that does not contain the word "duplicate" in ORDER NUMBER
df = df[~df["ORDER NUMBER"].astype(str).str.contains("duplicate", case=False, na=False)]
df

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,Prepaid,NaN
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,Prepaid,pending
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,Prepaid,success
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,Prepaid,success
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,Prepaid,success
...,...,...,...,...,...,...,...
7019,Return Exceptions,IN_PROGRESS,2025-06-12 12:18:19,RT183950955,700877196005,COD,NaN
7020,Return Exceptions,IN_PROGRESS,2025-06-12 16:30:09,RT183980671,701449142003,Prepaid,NaN
7021,Return Exceptions,IN_PROGRESS,2025-06-12 22:23:50,RT184041402,701558260004,Prepaid,NaN
7022,Return Exceptions,IN_PROGRESS,2025-06-12 14:40:38,RT183978459,700965211001,Prepaid,NaN


In [12]:
#Removing the rows where ITEM_ID contains the word "duplicate"
df = df[~df["ITEM_ID"].astype(str).str.contains("duplicate", case=False, na=False)]
df

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,Prepaid,NaN
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,Prepaid,pending
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,Prepaid,success
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,Prepaid,success
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,Prepaid,success
...,...,...,...,...,...,...,...
7019,Return Exceptions,IN_PROGRESS,2025-06-12 12:18:19,RT183950955,700877196005,COD,NaN
7020,Return Exceptions,IN_PROGRESS,2025-06-12 16:30:09,RT183980671,701449142003,Prepaid,NaN
7021,Return Exceptions,IN_PROGRESS,2025-06-12 22:23:50,RT184041402,701558260004,Prepaid,NaN
7022,Return Exceptions,IN_PROGRESS,2025-06-12 14:40:38,RT183978459,700965211001,Prepaid,NaN


## Need to Remove Alert

In [14]:
#Browsing the file from the system

#ntr_file = askopenfilename(
#    title="Select previous report data",
#    filetypes=[("Data files", "*.xlsx *.xls *.csv"), ("All files", "*.*")])
#
#if ntr_file.endswith('.csv'):
#    ntr_prev = pd.read_csv(ntr_file)
#elif ntr_file.endswith(('.xls', '.xlsx')):
#    df_ntr = pd.read_excel(ntr_file)
#else:
#    raise ValueError("Unsupported file format")
#
#ntr_prev.head()

In [15]:
#Reading the "Need to remove- order exception alert.xlsx" file

#NOTE - 1. Make sure that this file is stored at the same place where code is stored.
#       2. Make sure that file name("Need to remove- order exception alert") and extension (".xlsx") are same as mentioned or change them accordingly.

#TIP - Comment the below code and uncomment the above code to choose the above code to choose the file directly from system.

df_ntr = pd.read_excel("Need to remove- order exception alert.xlsx")
df_ntr.head()

,Order No,Action
0,EX0030251971,No action required as order created in sterling
1,EX0030253903,No action required as order created in sterling
2,EX0030256695,No action required as order created in sterling
3,EX0030274584,No action required as order created in sterling
4,EX0030276553,No action required as order created in sterling


In [16]:
#Removing the rows with ORDER NUMBER that matches with Order No in "Need to remove.." file.
df = df[~df["ORDER NUMBER"].isin(df_ntr["Order No"])]
df

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,Prepaid,NaN
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,Prepaid,pending
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,Prepaid,success
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,Prepaid,success
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,Prepaid,success
...,...,...,...,...,...,...,...
7019,Return Exceptions,IN_PROGRESS,2025-06-12 12:18:19,RT183950955,700877196005,COD,NaN
7020,Return Exceptions,IN_PROGRESS,2025-06-12 16:30:09,RT183980671,701449142003,Prepaid,NaN
7021,Return Exceptions,IN_PROGRESS,2025-06-12 22:23:50,RT184041402,701558260004,Prepaid,NaN
7022,Return Exceptions,IN_PROGRESS,2025-06-12 14:40:38,RT183978459,700965211001,Prepaid,NaN


In [17]:
#Copying the df into new df (df_ov) so any changes we make now will change in new df and our original df will not be affected.
df_ov = df.copy()

#Creating column "OV", after "ITEM_ID", by concatinating "ORDER NUMBER" and "ITEM_ID"
df_ov.insert(df.columns.get_loc("ITEM_ID") + 1, "OV", df["ORDER NUMBER"].astype(str) + df["ITEM_ID"].astype(str))
df_ov.head()

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,OV,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,EX0032235174443040205004,Prepaid,NaN
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,FN8257602445450161255002,Prepaid,pending
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,FN0156600541701560351002,Prepaid,success
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,FN0156600541701553228014,Prepaid,success
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,FN0156600541701481616004,Prepaid,success


In [18]:
#Removing rows with duplicate values in "OV"
df_ov.drop_duplicates(subset=["OV"], inplace=True)
df_ov

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,OV,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,EX0032235174443040205004,Prepaid,NaN
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,FN8257602445450161255002,Prepaid,pending
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,FN0156600541701560351002,Prepaid,success
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,FN0156600541701553228014,Prepaid,success
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,FN0156600541701481616004,Prepaid,success
...,...,...,...,...,...,...,...,...
7019,Return Exceptions,IN_PROGRESS,2025-06-12 12:18:19,RT183950955,700877196005,RT183950955700877196005,COD,NaN
7020,Return Exceptions,IN_PROGRESS,2025-06-12 16:30:09,RT183980671,701449142003,RT183980671701449142003,Prepaid,NaN
7021,Return Exceptions,IN_PROGRESS,2025-06-12 22:23:50,RT184041402,701558260004,RT184041402701558260004,Prepaid,NaN
7022,Return Exceptions,IN_PROGRESS,2025-06-12 14:40:38,RT183978459,700965211001,RT183978459700965211001,Prepaid,NaN


## Importing the Previous File shared by TOPS to do VLOOKUP

In [20]:
#Reading the previous report file from system.
prev_file = askopenfilename(
    title="Select previous report data",
    filetypes=[("Data files", "*.xlsx *.xls *.csv"), ("All files", "*.*")])

# Detect file extension and load accordingly
if prev_file.endswith('.csv'):
    df_prev = pd.read_csv(prev_file)
elif prev_file.endswith(('.xls', '.xlsx')):
    df_prev = pd.read_excel(prev_file)
else:
    raise ValueError("Unsupported file format")

df_prev.head()

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-05-30 14:43:45,FN7407607527,443032914003,Prepaid,success
1,Forward Exceptions,CREATED,2025-05-30 14:43:45,FN7407607527,442656493005,Prepaid,success
2,Forward Exceptions,CREATED,2025-06-08 18:24:53,FN6927606851,443281952003,Prepaid,success
3,Forward Exceptions,CREATED,2025-05-29 11:13:21,FN5082602008,701205527003,Prepaid,success
4,Forward Exceptions,CREATED,2025-05-19 21:54:49,FN1966606822,493685439,Prepaid,success


In [21]:
#Making the in previous file same as original file.
#TIP - If changes are already made in previous report file then you can comment the below 5 lines of code.

df_prev["ITEM_ID"] = df_prev["ITEM_ID"].apply(lambda x: str(Fraction(x).limit_denominator()) if pd.notna(x) else x)

In [22]:
df_prev = df_prev[~df_prev["ORDER NUMBER"].astype(str).str.contains("duplicate", case=False, na=False)]
df_prev = df_prev[~df_prev["ITEM_ID"].astype(str).str.contains("duplicate", case=False, na=False)]

In [23]:
df_prev = df_prev[~df_prev["ORDER NUMBER"].isin(df_ntr["Order No"])]

In [24]:
df_prev.insert(df_prev.columns.get_loc("ITEM_ID") + 1, "OV", df_prev["ORDER NUMBER"].astype(str) + df_prev["ITEM_ID"].astype(str))

In [25]:
df_prev.drop_duplicates(subset=["OV"], inplace=True)

In [26]:
df_prev

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,OV,PAYMENT MODE,PAYMENT STATUS
0,Forward Exceptions,CREATED,2025-05-30 14:43:45,FN7407607527,443032914003,FN7407607527443032914003,Prepaid,success
1,Forward Exceptions,CREATED,2025-05-30 14:43:45,FN7407607527,442656493005,FN7407607527442656493005,Prepaid,success
2,Forward Exceptions,CREATED,2025-06-08 18:24:53,FN6927606851,443281952003,FN6927606851443281952003,Prepaid,success
3,Forward Exceptions,CREATED,2025-05-29 11:13:21,FN5082602008,701205527003,FN5082602008701205527003,Prepaid,success
4,Forward Exceptions,CREATED,2025-05-19 21:54:49,FN1966606822,493685439,FN1966606822493685439,Prepaid,success
...,...,...,...,...,...,...,...,...
4731,Return Exceptions,REFUND_INITIATED,2025-06-11 14:44:46,RT183864643,443326109013,RT183864643443326109013,Prepaid,NaN
4732,Return Exceptions,IN_PROGRESS,2025-06-12 11:50:51,RT183962498,443067558002,RT183962498443067558002,COD,NaN
4733,Return Exceptions,IN_PROGRESS,2025-06-12 11:17:46,RT183970226,465366087001,RT183970226465366087001,Prepaid,NaN
4734,Return Exceptions,IN_PROGRESS,2025-06-12 08:17:34,RT183940862,700163190001,RT183940862700163190001,Prepaid,NaN


In [27]:
#Checking if the OV in Current report is available report or not. If available then returns TRUE else FALSE in the new column "Available in Previous Report"
df_ov["Available in Previous Report"] = df_ov["OV"].isin(df_prev["OV"])

#Marking TRUE as "Yes" and FALSE as "No"
df_ov["Available in Previous Report"] = df_ov["Available in Previous Report"].map({True: "Yes", False: "No"})
df_ov.head()

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,OV,PAYMENT MODE,PAYMENT STATUS,Available in Previous Report
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,EX0032235174443040205004,Prepaid,NaN,No
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,FN8257602445450161255002,Prepaid,pending,No
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,FN0156600541701560351002,Prepaid,success,Yes
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,FN0156600541701553228014,Prepaid,success,Yes
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,FN0156600541701481616004,Prepaid,success,Yes


## Creating PIVOT TABLE

In [29]:
#Copying into new df
df_ov_pivot = df_ov.copy()

In [30]:
#Converts TIMESTAMP to Date and removes errors, if any.
df_ov_pivot["CREATED TIMESTAMP"] = pd.to_datetime(df_ov_pivot["CREATED TIMESTAMP"], errors='coerce')

In [31]:
#Creating and importing "Month" and "Month-Day"
df_ov_pivot["Month"] = df_ov_pivot["CREATED TIMESTAMP"].dt.strftime("%B")
df_ov_pivot["Month-Day"] = df_ov_pivot["CREATED TIMESTAMP"].dt.strftime("%b-%d")
df_ov_pivot.head()

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,OV,PAYMENT MODE,PAYMENT STATUS,Available in Previous Report,Month,Month-Day
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,EX0032235174443040205004,Prepaid,NaN,No,June,Jun-12
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,FN8257602445450161255002,Prepaid,pending,No,June,Jun-12
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,FN0156600541701560351002,Prepaid,success,Yes,June,Jun-04
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,FN0156600541701553228014,Prepaid,success,Yes,June,Jun-04
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,FN0156600541701481616004,Prepaid,success,Yes,June,Jun-04


In [32]:
#Date Sorting

#Creating "Date" column and change name of months to numbers for sorting.
df_ov_pivot["Date"] = df_ov_pivot["CREATED TIMESTAMP"].dt.strftime("%Y-%m-%d")
sorted_order = df_ov_pivot.sort_values("Date")["Month-Day"].unique()
df_ov_pivot["Month-Day"] = pd.Categorical(df_ov_pivot["Month-Day"], categories=sorted_order, ordered=True)
df_ov_pivot.head()

,FLAG,STATUS,CREATED TIMESTAMP,ORDER NUMBER,ITEM_ID,OV,PAYMENT MODE,PAYMENT STATUS,Available in Previous Report,Month,Month-Day,Date
0,Forward Exceptions,CREATED,2025-06-12 19:36:01,EX0032235174,443040205004,EX0032235174443040205004,Prepaid,NaN,No,June,Jun-12,2025-06-12
1,Forward Exceptions,CREATED,2025-06-12 19:41:20,FN8257602445,450161255002,FN8257602445450161255002,Prepaid,pending,No,June,Jun-12,2025-06-12
2,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701560351002,FN0156600541701560351002,Prepaid,success,Yes,June,Jun-04,2025-06-04
3,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701553228014,FN0156600541701553228014,Prepaid,success,Yes,June,Jun-04,2025-06-04
4,Forward Exceptions,CREATED,2025-06-04 12:14:20,FN0156600541,701481616004,FN0156600541701481616004,Prepaid,success,Yes,June,Jun-04,2025-06-04


In [33]:
#Creating Pivot table
pivot_df = pd.pivot_table(
    df_ov_pivot,
    index=["Available in Previous Report", "FLAG"], #Rows
    columns="Month-Day",                            #Columns
    values="ORDER NUMBER",                          #Values
    aggfunc="count",                                
    fill_value=0,
    margins=True,
    margins_name="Grand Total"                      #Grand Total
).reset_index()

In [34]:
pivot_df

Month-Day,Available in Previous Report,FLAG,May-14,May-15,May-16,May-17,May-18,May-19,May-20,May-21,...,Jun-05,Jun-06,Jun-07,Jun-08,Jun-09,Jun-10,Jun-11,Jun-12,Jun-13,Grand Total
0,No,Forward Exceptions,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,104,2716,2820
1,No,Return Exceptions,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,27,964,991
2,Yes,Forward Exceptions,20,28,17,48,44,91,43,30,...,125,105,150,97,85,170,101,46,0,2453
3,Yes,Return Exceptions,0,0,0,0,0,0,0,0,...,28,26,35,28,20,32,28,10,0,377
4,Grand Total,,20,28,17,48,44,91,43,30,...,153,131,185,125,105,202,129,187,3680,6641


## Saving the Data

In [36]:
from datetime import datetime

In [37]:
# Generate filename with today's date
today_str = datetime.today().strftime("%d-%m-%y")
filename = f"AJIO Order Exceptions Alert- {today_str}.xlsx"

In [38]:
#Writing/Saving file as "AJIO Order Exceptions Alert-0 {today's date}.xlsx" with changed data in one sheet and pivot table in another sheet.
#NOTE - If 2 files are saved then the new file replaces the previous saved file if the name is not changed.
with pd.ExcelWriter(filename, engine="openpyxl") as writer:
    df_ov.to_excel(writer, sheet_name="AJIO Order Exceptions Alert-0", index=False)
    pivot_df.to_excel(writer, sheet_name="Pivot Table", index=False)